In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

In [2]:
review_data = pd.read_json('C:/Users/Vishal/Jupyter notebooks/Web Mining 1/Project/Magazine_Subscriptions.json', lines=True)
meta_data = pd.read_json('C:/Users/Vishal/Jupyter notebooks/Web Mining 1/Project/meta_Magazine_Subscriptions.json', lines=True)

In [3]:
#metadata
meta_data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Magazine Subscriptions, Professional & Educat...",,[REASON is edited for people interested in eco...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYLE, B01MCU84LB, B000UHI2LW, B01AKS14A...",,Reason Magazine,[],[],"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,NaT,,B00005N7NQ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",,[Written by and for musicians. Covers a variet...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYGE, B0054LRNC8, B000BVEELE, B00006KC3...",,String Letter Publishers,[],742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,NaT,,B00005N7OC,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",,[Allure is the beauty expert. Every issue is f...,,"<span class=""a-size-medium a-color-secondary""","[B001THPA4O, B002PXVZWW, B001THPA1M, B001THPA1...",,Conde Nast Publications,[],[],"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,NaT,,B00005N7OD,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,"[Magazine Subscriptions, Sports, Recreation & ...",,[FLIGHT JOURNAL includes articles on aviation ...,,"<span class=""a-size-medium a-color-secondary""","[B07JVF7QW4, B00ATQ6FPY, B002G551F6, B00008CGW...",,AirAge Publishing,[],[],"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,NaT,,B00005N7O9,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,"[Magazine Subscriptions, Professional & Educat...",,[RIDER is published for the road and street ri...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYD2, B01BM7TOU6, B000060MKJ, B000BNNIG...",,EPG Media & Specialty Information,[],[],"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,NaT,,B00005N7O6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [4]:
#preprocessing on metadata
meta_data_dc = meta_data.drop(['tech1', 'fit', 'tech2', 'main_cat', 'similar_item', 'date', 'price', 'imageURL', 'imageURLHighRes','also_view','also_buy','rank','details'], axis = 1)
meta_data_dc['category'] = meta_data_dc['category'].apply(lambda x: ','.join(map(str, x)))
meta_data_dc['description'] = meta_data_dc['description'].apply(lambda x: ','.join(map(str, x)))
meta_data_dc['feature'] = meta_data_dc['feature'].apply(lambda x: ','.join(map(str, x)))
meta_data_dc
meta_data_dc['metadata'] = meta_data_dc['category'] + ' ' + meta_data_dc['description'] + ' ' + meta_data_dc['title'] + ' ' + meta_data_dc['feature'] 
meta_data_1 = meta_data_dc[['asin', 'metadata']]
meta_data_f = meta_data_1.drop_duplicates()
meta_data_f

,asin,metadata
0,B00005N7NQ,"Magazine Subscriptions,Professional & Educatio..."
1,B00005N7OC,"Magazine Subscriptions,Arts, Music &amp; Photo..."
2,B00005N7OD,"Magazine Subscriptions,Fashion &amp; Style,Wom..."
3,B00005N7O9,"Magazine Subscriptions,Sports, Recreation & Ou..."
4,B00005N7O6,"Magazine Subscriptions,Professional & Educatio..."
...,...,...
3380,B01HI8V19K,"The game's most widely read publication, GOLF..."
3381,B01HI8V1I6,"Magazine Subscriptions,Sports, Recreation & Ou..."
3382,B01HI8V0ZK,"Magazine Subscriptions,Fashion & Style InStyle..."
3383,B01HI8V1C2,Sports Illustrated brings you spectacular act...


In [5]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
#     sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [6]:
meta_data_f['metadata']=meta_data_f['metadata'].map(lambda x:preprocess(x)) 

C:\Users\Vishal\AppData\Local\Temp/ipykernel_9632/2737106556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data_f['metadata']=meta_data_f['metadata'].map(lambda x:preprocess(x))


In [7]:
meta_data_f

,asin,metadata
0,B00005N7NQ,magazine subscriptions professional educationa...
1,B00005N7OC,magazine subscriptions arts music amp photogra...
2,B00005N7OD,magazine subscriptions fashion amp style women...
3,B00005N7O9,magazine subscriptions sports recreation outdo...
4,B00005N7O6,magazine subscriptions professional educationa...
...,...,...
3380,B01HI8V19K,game widely read publication golf magazine rea...
3381,B01HI8V1I6,magazine subscriptions sports recreation outdo...
3382,B01HI8V0ZK,magazine subscriptions fashion style instyle c...
3383,B01HI8V1C2,sports illustrated brings spectacular action p...


In [8]:
#reviewdata
review_data 

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN
1,5,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN
2,3,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN
3,5,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN
4,5,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89684,5,NaN,True,"08 16, 2016",AB5HCI50SF1TK,B01HI8V1C2,Katie Reid,This was a nice surprise for my boyfriend. He ...,Quick Delivery,1471305600,NaN,NaN
89685,1,NaN,True,"08 10, 2016",A2X11NZMPI7M0T,B01HI8V1C2,John,Magazine looks like it is printed on recycled ...,One Star,1470787200,NaN,NaN
89686,5,NaN,True,"07 31, 2016",A3EGFFBRQVO7L7,B01HI8V1C2,Jeff Johnson,cant go wrong with an SI subscription\nvery pl...,great magazine,1469923200,NaN,NaN
89687,5,NaN,True,"10 5, 2017",A3H23AVKFN62ZT,B01HIZSSQM,David Lashaway,This magazine is by far my all time favorite o...,Favorite metal detecting magazine,1507161600,NaN,NaN


In [9]:
#preprocessing on reviewdata
review_data_dc = review_data.drop(['verified', 'reviewTime', 'unixReviewTime', 'style', 'image'], axis = 1)
review_data_dc['r_metadata'] = review_data_dc['reviewText'] + ' ' + review_data_dc['summary']
review_data_f = review_data_dc[['overall', 'reviewerID', 'asin']]
review_data_f = review_data_f.drop_duplicates()
review_data_f

,overall,reviewerID,asin
0,5,AH2IFH762VY5U,B00005N7P0
1,5,AOSFI0JEYU4XM,B00005N7P0
2,3,A3JPFWKS83R49V,B00005N7OJ
3,5,A19FKU6JZQ2ECJ,B00005N7OJ
4,5,A25MDGOMZ2GALN,B00005N7P0
...,...,...,...
89684,5,AB5HCI50SF1TK,B01HI8V1C2
89685,1,A2X11NZMPI7M0T,B01HI8V1C2
89686,5,A3EGFFBRQVO7L7,B01HI8V1C2
89687,5,A3H23AVKFN62ZT,B01HIZSSQM


In [10]:
df_final = pd.merge(review_data_f, meta_data_f, on='asin')
df_final
# df_final1 = df_final.groupby('reviewerID').filter(lambda x: len(x) >= 10)
# df_final1

,overall,reviewerID,asin,metadata
0,5,AH2IFH762VY5U,B00005N7P0,magazine subscriptions technology computers am...
1,5,AOSFI0JEYU4XM,B00005N7P0,magazine subscriptions technology computers am...
2,5,A25MDGOMZ2GALN,B00005N7P0,magazine subscriptions technology computers am...
3,3,A3XT9XXWXFMJ1,B00005N7P0,magazine subscriptions technology computers am...
4,5,A3ERU005ES1IHT,B00005N7P0,magazine subscriptions technology computers am...
...,...,...,...,...
84036,5,A2XSSQH4VO5B1L,B01HI8V1C2,sports illustrated brings spectacular action p...
84037,5,A2K4AUMORPI9GZ,B01HI8V1C2,sports illustrated brings spectacular action p...
84038,5,AB5HCI50SF1TK,B01HI8V1C2,sports illustrated brings spectacular action p...
84039,1,A2X11NZMPI7M0T,B01HI8V1C2,sports illustrated brings spectacular action p...


In [11]:
#removing stopwords and creating tfidfvector for metadata
tfidf = TfidfVectorizer(smooth_idf = False, sublinear_tf = True)
tfidf_matrix = tfidf.fit_transform(meta_data_f['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=meta_data_f.asin.tolist())
print(tfidf_df.shape)
tfidf_df

(2320, 9008)


,0,1,2,3,4,5,6,7,8,9,...,8998,8999,9000,9001,9002,9003,9004,9005,9006,9007
B00005N7NQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7OD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7O9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7O6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01HI8V19K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01HI8V1I6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01HI8V0ZK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01HI8V1C2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(content_correlation.shape)
content_correlation
# sim_df=pd.DataFrame(content_correlation, index = tfidf_df.index,  columns=[meta_data_f.asin])
# sim_df

(2320, 2320)


array([[1.        , 0.01059682, 0.0252296 , ..., 0.06946933, 0.01252672,
        0.0065548 ],
       [0.01059682, 1.        , 0.09224949, ..., 0.01568643, 0.04907142,
        0.02760276],
       [0.0252296 , 0.09224949, 1.        , ..., 0.11155794, 0.01625149,
        0.11945348],
       ...,
       [0.06946933, 0.01568643, 0.11155794, ..., 1.        , 0.0852351 ,
        0.11440766],
       [0.01252672, 0.04907142, 0.01625149, ..., 0.0852351 , 1.        ,
        0.09720575],
       [0.0065548 , 0.02760276, 0.11945348, ..., 0.11440766, 0.09720575,
        1.        ]])

In [13]:
ratings = pd.pivot_table(df_final, values='overall', index=['reviewerID'], columns = ['asin']).fillna(0)
ratings

asin,B00005N7NQ,B00005N7O3,B00005N7O4,B00005N7O6,B00005N7O9,B00005N7OA,B00005N7OC,B00005N7OD,B00005N7OF,B00005N7OJ,...,B01HF4AQE8,B01HF4AQIE,B01HI8V0ZK,B01HI8V10E,B01HI8V11I,B01HI8V19K,B01HI8V1AE,B01HI8V1C2,B01HI8V1I6,B01HI8V1MC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0068533X8Y5TYUJWWIC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A02292976AOUT3I4ZLFA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0333047WGK24IZKLDP2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0479411TZTHITJ9TFB8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0534350D18UHJKPKZ1W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZS7W015QH2L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZZVFTH1QONVD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZZXJAE2DILET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def predict(ratings):
    pred = ratings.dot(content_correlation) / np.array([np.abs(content_correlation).sum(axis=1)])
    return pred

In [17]:
pred_r = predict(ratings)
pred_r

,0,1,2,3,4,5,6,7,8,9,...,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0068533X8Y5TYUJWWIC,0.001645,0.000718,0.002728,0.000869,0.000471,0.002829,0.002305,0.001118,0.003288,0.000922,...,0.001724,0.001724,0.001463,0.000318,0.000178,0.002301,0.000854,0.000747,0.000166,0.001862
A02292976AOUT3I4ZLFA,0.000356,0.000685,0.002901,0.000464,0.001935,0.003159,0.007269,0.000504,0.000533,0.000608,...,0.001345,0.001345,0.003079,0.000385,0.005566,0.001100,0.002109,0.000819,0.000435,0.001979
A0333047WGK24IZKLDP2,0.000308,0.000793,0.000542,0.000464,0.000446,0.001629,0.001304,0.000217,0.001781,0.001170,...,0.001222,0.001222,0.002487,0.001432,0.001213,0.000507,0.000859,0.000523,0.000187,0.000786
A0479411TZTHITJ9TFB8,0.001146,0.001748,0.003322,0.000747,0.001118,0.002832,0.004396,0.001433,0.002574,0.001119,...,0.002292,0.002292,0.001666,0.001069,0.002666,0.002591,0.001644,0.001594,0.001880,0.004746
A0534350D18UHJKPKZ1W,0.001774,0.000947,0.000598,0.000640,0.001138,0.000666,0.005383,0.002329,0.001268,0.001932,...,0.000418,0.000418,0.000640,0.001718,0.001550,0.000834,0.000697,0.000639,0.000669,0.000894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZS7W015QH2L,0.002695,0.002274,0.002596,0.000768,0.003112,0.000798,0.001642,0.003815,0.001581,0.002057,...,0.000501,0.000501,0.001649,0.000709,0.001792,0.001000,0.000835,0.001373,0.000802,0.001033
AZZVFTH1QONVD,0.001417,0.000597,0.002195,0.000404,0.002371,0.003174,0.001780,0.002007,0.001213,0.000509,...,0.000960,0.000960,0.000404,0.000373,0.000451,0.004024,0.001731,0.001158,0.000422,0.000260
AZZXJAE2DILET,0.002899,0.000603,0.000381,0.001358,0.001653,0.000424,0.000513,0.002026,0.001262,0.000881,...,0.001136,0.001136,0.001945,0.000377,0.000456,0.001093,0.000831,0.000675,0.001212,0.001177


In [18]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 5))
pred_r_scaled = min_max_scaler.fit_transform(pred_r)
pred_r_scaled_df = pd.DataFrame(pred_r_scaled, index = ratings.index,  columns=[meta_data_f.asin] )
pred_r_scaled_df

asin,B00005N7NQ,B00005N7OC,B00005N7OD,B00005N7O9,B00005N7O6,B00005N7P0,B00005N7QG,B00005N7PI,B00005N7OP,B00005N7Q5,...,B01HF4AQIE,B01HF4AQE8,B01HI8V11I,B01HI8V1AE,B01HI8V10E,B01HI8V19K,B01HI8V1I6,B01HI8V0ZK,B01HI8V1C2,B01HI8V1MC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0068533X8Y5TYUJWWIC,0.102402,0.053417,0.150388,0.043374,0.031991,0.135926,0.107889,0.075083,0.189174,0.042562,...,0.076251,0.076251,0.063088,0.020732,0.011772,0.142691,0.042888,0.051549,0.012424,0.087924
A02292976AOUT3I4ZLFA,0.022184,0.050954,0.159932,0.023143,0.131349,0.151772,0.340212,0.033877,0.030675,0.028040,...,0.059478,0.059478,0.132816,0.025107,0.368231,0.068193,0.105905,0.056530,0.032527,0.093415
A0333047WGK24IZKLDP2,0.019166,0.058980,0.029897,0.023163,0.030285,0.078293,0.061048,0.014563,0.102471,0.053975,...,0.054033,0.054033,0.107268,0.093415,0.080263,0.031423,0.043123,0.036051,0.013982,0.037102
A0479411TZTHITJ9TFB8,0.071370,0.129967,0.183112,0.037291,0.075922,0.136061,0.205742,0.096274,0.148082,0.051658,...,0.101370,0.101370,0.071883,0.069750,0.176365,0.160712,0.082577,0.109936,0.140476,0.224081
A0534350D18UHJKPKZ1W,0.110472,0.070383,0.032991,0.031967,0.077263,0.031988,0.251944,0.156440,0.072945,0.089167,...,0.018483,0.018483,0.027614,0.112084,0.102552,0.051730,0.035009,0.044065,0.050000,0.042207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZS7W015QH2L,0.167820,0.169114,0.143113,0.038314,0.211271,0.038339,0.076875,0.256273,0.090944,0.094919,...,0.022153,0.022153,0.071130,0.046257,0.118513,0.062002,0.041960,0.094742,0.059927,0.048776
AZZVFTH1QONVD,0.088261,0.044366,0.121029,0.020151,0.160977,0.152511,0.083300,0.134782,0.069774,0.023496,...,0.042449,0.042449,0.017407,0.024328,0.029866,0.249560,0.086948,0.079920,0.031517,0.012292
AZZXJAE2DILET,0.180515,0.044803,0.021001,0.067814,0.112208,0.020362,0.024018,0.136109,0.072604,0.040678,...,0.050249,0.050249,0.083896,0.024568,0.030160,0.067811,0.041741,0.046595,0.090521,0.055578


In [19]:
def recommend_products(userID, num_recommendations):
    sorted_user_predictions = pred_r_scaled_df.loc[userID].sort_values(ascending=False)
    
    user_data = review_data_f[review_data_f.reviewerID == (userID)]
    
    user_full = (user_data.merge(meta_data_f).sort_values(['overall'], ascending=False))
    
    recommendations = (meta_data_f[~meta_data_f['asin'].isin(user_full['asin'])].
                   merge(pd.DataFrame(sorted_user_predictions).rename_axis('asin').reset_index()).
                   sort_values(userID, ascending = False).
                   rename(columns = {userID:'Content based ratings'})
                      )
    recommendations = recommendations[:num_recommendations]
    return user_full, recommendations

In [20]:
user_full , recommendations = recommend_products('AZZXJAE2DILET', 10)
recommendations

,asin,metadata,Content based ratings
217,B000060MJC,magazine subscriptions professional educationa...,1.982700
426,B00006K8UR,magazine subscriptions professional educationa...,0.836829
1209,B000FI91VG,magazine subscriptions religion spirituality c...,0.803300
735,B00007B0OR,magazine subscriptions religion spirituality c...,0.803300
1153,B000BT4GR8,magazine subscriptions religion spirituality c...,0.803300
1499,B00529YCWQ,magazine subscriptions religion spirituality c...,0.803300
927,B00008RUG1,magazine subscriptions religion spirituality c...,0.803300
212,B000060MH4,magazine subscriptions professional educationa...,0.794039
402,B00006KCWX,magazine subscriptions professional educationa...,0.785126
417,B00006KM3Q,magazine subscriptions professional educationa...,0.723760


In [21]:
pred_ratings = recommendations['Content based ratings']
user_ratings = user_full['overall']

pred_ratings1 = pred_ratings[:len(user_ratings)]
user_ratings1 = user_ratings[:len(pred_ratings)]
user_ratings

# # # #RMSE
RMSE = np.sqrt(mean_squared_error(user_ratings1, pred_ratings1))
RMSE

1.0172996356107131

In [22]:
# # #MAE
MAE = mean_absolute_error(user_ratings1, pred_ratings1)
MAE

1.0172996356107131

In [23]:
#predicting rating using SVM

df_final = df_final.sort_values(['asin'], ascending=False)
df_final
X = df_final['metadata'].copy()
X 

y = df_final['overall'].copy()
y
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [24]:
y_test

78223    5
54965    5
22213    3
9770     5
23087    5
        ..
6415     5
5255     4
42495    5
33342    3
47998    5
Name: overall, Length: 21011, dtype: int64

In [25]:
pipe = Pipeline([('Vectorizer', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('SVM classifier', SVC())])

pipe.fit(X_train, y_train)

Pipeline(steps=[('Vectorizer', CountVectorizer()),
                ('tfidf', TfidfTransformer()), ('SVM classifier', SVC())])

In [ ]:
# y_pred = pipe.predict(X_test)

In [ ]:
# y_pred

In [ ]:
df_final
df_final.groupby(['overall']).agg({'overall':'count'})

In [26]:
pred_all = pipe.predict(X)

In [27]:
pred_all

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [28]:
RMSE = np.sqrt(mean_squared_error(y, pred_all))
RMSE

1.6963862613962728

In [29]:
pred_all_df = pd.DataFrame(pred_all, index=df_final.index, columns = ['Predicted ratings'] )

pred_all_df
# pred_all_df.groupby(['Predicted ratings']).agg({'Predicted ratings':'count'})

,Predicted ratings
78842,5
78837,5
78844,5
78843,5
78841,5
...,...
5240,5
5242,5
5243,5
5244,5


In [30]:
df_final_w_pred = df_final.merge(pred_all_df, on = df_final.index)
df_final_pred = df_final_w_pred[['reviewerID', 'asin', 'Predicted ratings']]
df_final_pred

# df_final_pred.loc[df_final_pred['reviewerID'] == 'AZZXJAE2DILET']

,reviewerID,asin,Predicted ratings
0,A2S7CNM09VOXD2,B01HI8V1MC,5
1,ASU7EOBD3Y4BV,B01HI8V1MC,5
2,AYZBIIGR7IUPC,B01HI8V1MC,5
3,A3BHMLC4E0GOLC,B01HI8V1MC,5
4,A1UM4RQKYWUV1K,B01HI8V1MC,5
...,...,...,...
84036,AJMYX6BWDEHZA,B00005N7NQ,5
84037,A16EVZQORWF6HR,B00005N7NQ,5
84038,ATYV9NMHGBYAO,B00005N7NQ,5
84039,A3HJHWCJUI4RTQ,B00005N7NQ,5


In [31]:
SVM_ratings = pd.pivot_table(df_final_pred, values='Predicted ratings', index=['reviewerID'], columns = ['asin']).fillna(0)
pred_ratings = predict(SVM_ratings)
pred_ratings

,0,1,2,3,4,5,6,7,8,9,...,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0068533X8Y5TYUJWWIC,0.001645,0.000718,0.002728,0.000869,0.000471,0.002829,0.002305,0.001118,0.003288,0.000922,...,0.001724,0.001724,0.001463,0.000318,0.000178,0.002301,0.000854,0.000747,0.000166,0.001862
A02292976AOUT3I4ZLFA,0.000356,0.000685,0.002901,0.000464,0.001935,0.003159,0.007269,0.000504,0.000533,0.000608,...,0.001345,0.001345,0.003079,0.000385,0.005566,0.001100,0.002109,0.000819,0.000435,0.001979
A0333047WGK24IZKLDP2,0.000385,0.000992,0.000678,0.000580,0.000558,0.002037,0.001630,0.000271,0.002226,0.001462,...,0.001527,0.001527,0.003108,0.001790,0.001517,0.000633,0.001073,0.000653,0.000234,0.000982
A0479411TZTHITJ9TFB8,0.001146,0.001748,0.003322,0.000747,0.001118,0.002832,0.004396,0.001433,0.002574,0.001119,...,0.002292,0.002292,0.001666,0.001069,0.002666,0.002591,0.001644,0.001594,0.001880,0.004746
A0534350D18UHJKPKZ1W,0.001774,0.000947,0.000598,0.000640,0.001138,0.000666,0.005383,0.002329,0.001268,0.001932,...,0.000418,0.000418,0.000640,0.001718,0.001550,0.000834,0.000697,0.000639,0.000669,0.000894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZS7W015QH2L,0.002695,0.002274,0.002596,0.000768,0.003112,0.000798,0.001642,0.003815,0.001581,0.002057,...,0.000501,0.000501,0.001649,0.000709,0.001792,0.001000,0.000835,0.001373,0.000802,0.001033
AZZVFTH1QONVD,0.001417,0.000597,0.002195,0.000404,0.002371,0.003174,0.001780,0.002007,0.001213,0.000509,...,0.000960,0.000960,0.000404,0.000373,0.000451,0.004024,0.001731,0.001158,0.000422,0.000260
AZZXJAE2DILET,0.004832,0.001004,0.000635,0.002264,0.002755,0.000706,0.000855,0.003377,0.002103,0.001469,...,0.001894,0.001894,0.003242,0.000628,0.000760,0.001822,0.001385,0.001126,0.002020,0.001962


In [32]:
SVM_pred_r_scaled = min_max_scaler.fit_transform(pred_ratings)
SVM_pred_r_scaled_df = pd.DataFrame(SVM_pred_r_scaled, index = ratings.index,  columns=[meta_data_f.asin] )
SVM_pred_r_scaled_df

asin,B00005N7NQ,B00005N7OC,B00005N7OD,B00005N7O9,B00005N7O6,B00005N7P0,B00005N7QG,B00005N7PI,B00005N7OP,B00005N7Q5,...,B01HF4AQIE,B01HF4AQE8,B01HI8V11I,B01HI8V1AE,B01HI8V10E,B01HI8V19K,B01HI8V1I6,B01HI8V0ZK,B01HI8V1C2,B01HI8V1MC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0068533X8Y5TYUJWWIC,0.090478,0.039866,0.110408,0.040346,0.024384,0.101122,0.083448,0.054164,0.142778,0.030454,...,0.070558,0.070558,0.061802,0.015761,0.009092,0.138526,0.032898,0.038463,0.009335,0.066122
A02292976AOUT3I4ZLFA,0.019601,0.038028,0.117414,0.021527,0.100119,0.112910,0.263139,0.024438,0.023152,0.020063,...,0.055037,0.055037,0.130109,0.019088,0.284384,0.066202,0.081236,0.042180,0.024440,0.070252
A0333047WGK24IZKLDP2,0.021168,0.055023,0.027436,0.026932,0.028856,0.072808,0.059023,0.013132,0.096674,0.048275,...,0.062498,0.062498,0.131352,0.088773,0.077483,0.038133,0.041347,0.033624,0.013133,0.034878
A0479411TZTHITJ9TFB8,0.063060,0.096997,0.134432,0.034688,0.057871,0.101222,0.159133,0.069450,0.111764,0.036962,...,0.093800,0.093800,0.070418,0.053028,0.136206,0.156021,0.063342,0.082028,0.105551,0.168518
A0534350D18UHJKPKZ1W,0.097609,0.052528,0.024220,0.029736,0.058892,0.023797,0.194868,0.112853,0.055055,0.063800,...,0.017103,0.017103,0.027051,0.085212,0.079200,0.050221,0.026854,0.032879,0.037569,0.031741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZS7W015QH2L,0.148279,0.126214,0.105067,0.035640,0.161038,0.028523,0.059459,0.184871,0.068639,0.067916,...,0.020499,0.020499,0.069681,0.035167,0.091528,0.060192,0.032186,0.070691,0.045028,0.036681
AZZVFTH1QONVD,0.077984,0.033111,0.088854,0.018744,0.122702,0.113460,0.064429,0.097229,0.052661,0.016812,...,0.039280,0.039280,0.017052,0.018495,0.023065,0.242276,0.066695,0.059632,0.023682,0.009244
AZZXJAE2DILET,0.265826,0.055729,0.025696,0.105133,0.142548,0.025248,0.030961,0.163644,0.091328,0.048509,...,0.077495,0.077495,0.136977,0.031129,0.038820,0.109720,0.053364,0.057944,0.113359,0.069661


In [33]:
def recommend_products_SVM(userID, num_recommendations):
    sorted_user_predictions = SVM_pred_r_scaled_df.loc[userID].sort_values(ascending=False)
    
    user_data = review_data_f[review_data_f.reviewerID == (userID)]
    
    user_full = (user_data.merge(meta_data_f).sort_values(['overall'], ascending=False))
    
    recommendations = (meta_data_f[~meta_data_f['asin'].isin(user_full['asin'])].
                   merge(pd.DataFrame(sorted_user_predictions).rename_axis('asin').reset_index()).
                   sort_values(userID, ascending = False).
                   rename(columns = {userID:'Content based ratings'})
                      )
    recommendations = recommendations[:num_recommendations]
    return user_full, recommendations

In [36]:
user_full_svm , svm_recommendations = recommend_products_SVM('AZZXJAE2DILET', 10)
svm_recommendations

,asin,metadata,Content based ratings
217,B000060MJC,magazine subscriptions professional educationa...,2.422945
426,B00006K8UR,magazine subscriptions professional educationa...,1.207751
212,B000060MH4,magazine subscriptions professional educationa...,1.204445
523,B00006KWSP,magazine subscriptions professional educationa...,1.184404
402,B00006KCWX,magazine subscriptions professional educationa...,1.110824
735,B00007B0OR,magazine subscriptions religion spirituality c...,0.983790
1499,B00529YCWQ,magazine subscriptions religion spirituality c...,0.983790
927,B00008RUG1,magazine subscriptions religion spirituality c...,0.983790
1209,B000FI91VG,magazine subscriptions religion spirituality c...,0.983790
1153,B000BT4GR8,magazine subscriptions religion spirituality c...,0.983790


In [37]:
RMSE = np.sqrt(mean_squared_error(y, pred_all))
RMSE

1.6963862613962728

In [38]:
MAE = mean_absolute_error(y, pred_all)
MAE

0.9456693756618794